In [187]:
import pandas as pd
import numpy as np

In [188]:
order = 5
states = 4


transitionProbabilities = np.ndarray((states, states))
transitionProbabilities[:] = 0

transitionCounts = np.ndarray((states, states))
transitionCounts[:] = 0

transitionStateCounts = np.zeros(states)



In [189]:
def jsonToDataFrame(stockData):
    opens = []
    highs = []
    lows = []
    closes = []
    volumes = []
    dates = []

    for key in stockData:
        (open_, high, low, close, volume) = stockData[key]
        opens.append(open_)
        highs.append(high)
        lows.append(low)
        closes.append(close)
        volumes.append(volume)
        dates.append(key)

    data = {'Date':dates, 'Open':opens, 'High':highs, 'Low':lows, 'Close':closes, 'Volume': volumes}

    return pd.DataFrame(data=data)

In [205]:



def convertArrayToTimeSeries(array, duration):
    length = len(array) - duration
    if(length <= 0):
        return np.ndarray((0,duration))

    subTimeSeries = np.ndarray((length, duration))
    for i in range(length):
        subTimeSeries[i] = array[i:i+duration]
    return subTimeSeries

def normalizeTimeSeries(timeSeries):
    avg = np.mean(timeSeries, axis=1)
    var = np.var(timeSeries, axis=1)
    

    timeSeriesNorm = np.ndarray(timeSeries.shape)
    for i in range(len(timeSeries)):
        if(var[i] == 0):
            timeSeriesNorm[i] = np.zeros(timeSeries[i].shape)
        else:
            timeSeriesNorm[i] = (timeSeries[i] - avg[i])/var[i]
    return timeSeriesNorm




In [206]:
from convertStockToJson import getJsonDataFromFile

def loadTimeSeriesNormalized(name):
    stockData = getJsonDataFromFile(name)
    stockDatadf = jsonToDataFrame(stockData) 
    subTimeSeries = convertArrayToTimeSeries(stockDatadf['Close'], order)
    return normalizeTimeSeries(subTimeSeries)


In [207]:
subTimeSeriesNorm = loadTimeSeriesNormalized('VAS')

here


In [208]:
from sklearn.cluster import KMeans
import numpy as np
import numpy as np
import matplotlib.pyplot as plt




kmeans = KMeans(n_clusters=states, random_state=0).fit(subTimeSeriesNorm)

for cluster in kmeans.cluster_centers_:
   
    plt.plot(cluster)
    plt.show()






ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [175]:
subTimeSeriesClassified = kmeans.predict(subTimeSeriesNorm)

In [176]:

def trainMarkokChain(subSeriesClassifified):
    for i in range(len(subSeriesClassifified)-1):
        currentState = subSeriesClassifified[i]
        nextState = subSeriesClassifified[i+1]
        
        transitionCounts[currentState, nextState] += 1
        transitionStateCounts[currentState] += 1


trainMarkokChain(subTimeSeriesClassified)


print(transitionCounts)
print(transitionStateCounts)

[[ 15.  14.  99.   1.]
 [ 68. 246.  10.  48.]
 [ 22.  11. 249.  76.]
 [ 23. 102.   0.  23.]]
[129. 372. 358. 148.]


In [177]:
def updateProbabilities():
    for i in range(transitionProbabilities.shape[0]):
        if(transitionStateCounts[i] != 0):
            for j in range(transitionProbabilities.shape[1]):
                transitionProbabilities[i, j] = transitionCounts[i, j]/transitionStateCounts[i]

updateProbabilities()
print(transitionProbabilities)

[[0.11627907 0.10852713 0.76744186 0.00775194]
 [0.1827957  0.66129032 0.02688172 0.12903226]
 [0.06145251 0.03072626 0.69553073 0.2122905 ]
 [0.15540541 0.68918919 0.         0.15540541]]


In [178]:
def loadTimeSeriesClassified(name):
    normalized = loadTimeSeriesNormalized(name)
    if(normalized.shape[0] > 0):
        return (kmeans.predict(normalized), True)
    else:
        return (None, False)

In [179]:
def testMarkovChain(subSeriesClassifified):
    numCorrect = 0
    length = len(subSeriesClassifified)-1
    for i in range(length):
        currentState = subSeriesClassifified[i]
        nextStateCorrect = subSeriesClassifified[i+1]
        nextStatePredict = np.argmax(transitionStateCounts[currentState])
        if(nextStatePredict == nextStateCorrect):
            numCorrect += 1
    if(length > 0):
        return numCorrect/length
    else:
        return None

In [180]:
from filesInPath import filesInPath

filenames = filesInPath('data')
import random

# random.shuffle(filenames)
# filenames = filenames[:40]
filenames = ['VAS', 'IOO', 'XRO']

for (fileName,i) in zip(filenames, range(len(filenames))):
    #fileName = fileName[:-5]
 
    (timeSeriesClassified, success) = loadTimeSeriesClassified(fileName)  
    if(success):  
        percetage = testMarkovChain(timeSeriesClassified)  
        if(percetage != None):
            print(fileName + ",  " + str(percetage * 100 ) + "%")


VAS,  12.711022840119165%
IOO,  10.526315789473683%
XRO,  14.115308151093439%
